<a href="https://colab.research.google.com/github/johnsonAyo/esemble_model/blob/main/ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loan Dataset
## Goal of Analysis: Analyzing the loan dataset using Decision Tree, Random Forest and Ensemble of Decision tree Tree and Random Forest.

In [ ]:
#Imports
import pandas as pd
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, StackingClassifier

# Data Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
loan_df = pd.read_csv("gdrive/My Drive/Colab Notebooks/loan_dataset.csv")
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


## Data Exploration

In [ ]:
loan_df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [ ]:
!pip install ydata-profiling==4.6.2
from ydata_profiling import ProfileReport
# Search for overall trends in the dataset
ProfileReport(loan_df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Data Cleaning

In [ ]:
#Missing fields [gender, married, dependents, self_employed, loan_amount_term, credit_history, loan_amount]
loan_df['Gender'].fillna("Male", inplace=True)
loan_df['Married'].fillna("Yes", inplace=True)
loan_df['Dependents'].fillna("0", inplace=True)
loan_df['Self_Employed'].fillna("No", inplace=True)
loan_df['Loan_Amount_Term'].fillna(360, inplace=True)
loan_df['Credit_History'].fillna(1, inplace=True)
loan_df['LoanAmount'].fillna(loan_df['LoanAmount'].mean(), inplace=True)
loan_df.drop('Loan_ID', axis=1, inplace=True)

In [ ]:
le = preprocessing.LabelEncoder()
gender_cat = le.fit_transform(loan_df.Gender)
married_cat = le.fit_transform(loan_df.Married)
dependents_cat = le.fit_transform(loan_df.Dependents)
education_cat = le.fit_transform(loan_df.Education)
self_employed_cat = le.fit_transform(loan_df.Self_Employed)
property_area_cat = le.fit_transform(loan_df.Property_Area)
loan_status_cat = le.fit_transform(loan_df.Loan_Status)

loan_df['gender_cat'] = gender_cat
loan_df['married_cat'] = married_cat
loan_df['dependents_cat'] = dependents_cat
loan_df['education_cat'] = education_cat
loan_df['self_employed_cat'] = self_employed_cat
loan_df['property_area_cat'] = property_area_cat
loan_df['loan_status_cat'] = loan_status_cat

dummy_fields = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']
data = loan_df.drop(dummy_fields, axis = 1)
data = loan_df.reindex(['gender_cat', 'married_cat', 'dependents_cat', 'education_cat', 'self_employed_cat', 'property_area_cat', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'loan_status_cat'], axis=1)
data

,gender_cat,married_cat,dependents_cat,education_cat,self_employed_cat,property_area_cat,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,loan_status_cat
0,1,0,0,0,0,2,5849,0.0,146.412162,360.0,1.0,1
1,1,1,1,0,0,0,4583,1508.0,128.000000,360.0,1.0,0
2,1,1,0,0,1,2,3000,0.0,66.000000,360.0,1.0,1
3,1,1,0,1,0,2,2583,2358.0,120.000000,360.0,1.0,1
4,1,0,0,0,0,2,6000,0.0,141.000000,360.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,0,2900,0.0,71.000000,360.0,1.0,1
610,1,1,3,0,0,0,4106,0.0,40.000000,180.0,1.0,1
611,1,1,1,0,0,2,8072,240.0,253.000000,360.0,1.0,1
612,1,1,2,0,0,2,7583,0.0,187.000000,360.0,1.0,1


In [ ]:
data[data.isnull().any(axis=1)].head()

,gender_cat,married_cat,dependents_cat,education_cat,self_employed_cat,property_area_cat,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,loan_status_cat


## Models

### Test Train Split

In [ ]:
#Split inputs and output
X = data.iloc[:, 0:11]
Y = data.iloc[:, 11]

In [ ]:
#Test/Train Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train

,gender_cat,married_cat,dependents_cat,education_cat,self_employed_cat,property_area_cat,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
415,0,0,0,0,0,2,2995,0.0,60.0,360.0,1.0
3,1,1,0,1,0,2,2583,2358.0,120.0,360.0,1.0
343,1,1,3,1,0,1,3173,0.0,74.0,360.0,1.0
569,1,1,0,0,0,2,3166,2064.0,104.0,360.0,0.0
335,1,1,0,0,1,1,5503,4490.0,70.0,360.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
205,0,0,0,1,0,1,4408,0.0,120.0,360.0,1.0
296,1,1,1,0,0,1,6875,0.0,200.0,360.0,1.0
12,1,1,2,0,0,2,3073,8106.0,200.0,360.0,1.0
221,1,1,2,0,0,1,4009,1717.0,116.0,360.0,1.0


### Random Forest

In [ ]:
#RF1 - All attributes
#Initalize + fit model
clf = RandomForestClassifier(n_jobs=2, random_state=0).fit(X_train, y_train)

#Predictions
y_pred = clf.predict(X_test)

In [ ]:
rf_report = classification_report(y_test, y_pred)
print('Random Forest Report:')
print(rf_report)

Random Forest Report:
              precision    recall  f1-score   support

           0       0.88      0.41      0.56        34
           1       0.81      0.98      0.89        89

    accuracy                           0.82       123
   macro avg       0.84      0.69      0.72       123
weighted avg       0.83      0.82      0.80       123



### Stacking

In [ ]:
# Define the base models
base_models = [
    ('decision_tree', DecisionTreeClassifier()),
    ('random_forest', RandomForestClassifier()),
    ('svm', SVC())
]

# Create the stacking ensemble classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Fit the stacking model on the training data
stacking_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = stacking_model.predict(X_test)


In [ ]:
stacking_report = classification_report(y_test, y_pred)
print('Stacking Ensemble Report:')
print(stacking_report)

Stacking Ensemble Report:
              precision    recall  f1-score   support

           0       0.88      0.41      0.56        34
           1       0.81      0.98      0.89        89

    accuracy                           0.82       123
   macro avg       0.84      0.69      0.72       123
weighted avg       0.83      0.82      0.80       123



### Boosting

In [ ]:
# Create the boosting ensemble classifier
boosting_model = AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=50)

# Fit the boosting model on the training data
boosting_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred_boosting = boosting_model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
boosting_report = classification_report(y_test, y_pred_boosting)
print('Boosting Ensemble Report:')
print(boosting_report)

Boosting Ensemble Report:
              precision    recall  f1-score   support

           0       0.76      0.38      0.51        34
           1       0.80      0.96      0.87        89

    accuracy                           0.80       123
   macro avg       0.78      0.67      0.69       123
weighted avg       0.79      0.80      0.77       123



### Bagging

In [ ]:
# Create the bagging ensemble classifier
bagging_model = BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=10)

# Fit the bagging model on the training data
bagging_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred_bagging = bagging_model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
bagging_report = classification_report(y_test, y_pred_bagging)
print('Bagging Ensemble Report:')
print(bagging_report)

Bagging Ensemble Report:
              precision    recall  f1-score   support

           0       0.93      0.41      0.57        34
           1       0.81      0.99      0.89        89

    accuracy                           0.83       123
   macro avg       0.87      0.70      0.73       123
weighted avg       0.85      0.83      0.80       123

